This benchmark shows advantages of using bcp over odbc. It is quite crude, however, and in practice the results may vary strongly depending on the number/kind of indexes on a table, the connectivity to the database and the difference in scale between the ETL server and SQL Server among other factors. That being said, I would expect similar tendencies.

# Imports & Connection setup

In [ ]:
%pip install pyodbc plotly arrow-odbc

In [2]:
import arrow_bcp as ab
import pyarrow as pa
import pyodbc
from plotly import graph_objects as go
import random
import datetime
from decimal import Decimal
import time
import timeit

Database connection details need to be filled in

In [3]:
SQL_URL = "##########"
SQL_DB = "##########"
SQL_LOGIN = "##########"
SQL_PWD = "##########"

In [4]:
NR_ROWS = 200_000
NR_BATCHES = 1
NR_REPS = 5

In [5]:
pyodbc_con = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SQL_URL};DATABASE={SQL_DB};UID={SQL_LOGIN};PWD={SQL_PWD}"
)
bcp_con = ab.ConnectionInfo(
    (f"-S tcp:{SQL_URL},1433 -d {SQL_DB} -U {SQL_LOGIN} -P {SQL_PWD}").split()
)
bcp_con_tablock = ab.ConnectionInfo(
    (f"-S tcp:{SQL_URL},1433 -d {SQL_DB} -U {SQL_LOGIN} -P {SQL_PWD} -h TABLOCK").split()
)

# Generate random data

In [6]:
alphabet = [
    chr(code_point) for current_range in [
        (0x0021, 0x0021),
        (0x0023, 0x0026),
        (0x0028, 0x007E),
        (0x00A1, 0x00AC),
        (0x00AE, 0x00FF),
        (0x0100, 0x017F),
        (0x0180, 0x024F),
        (0x2C60, 0x2C7F),
        (0x16A0, 0x16F0),
        (0x0370, 0x0377),
        (0x037A, 0x037E),
        (0x0384, 0x038A),
        (0x038C, 0x038C),
    ]
    for code_point in range(current_range[0], current_range[1] + 1)
]
def random_utf8(length):
    return ''.join(random.choices(alphabet, k=length))

dmin, dmax = pa.array([datetime.date(1,1,1), datetime.date(9999,12,31)], type=pa.date32()).cast(pa.int32()).to_pylist()
tmin, tmax = pa.array([0, datetime.time(23,59,59,999999)], type=pa.time64('us')).cast(pa.int64()).to_pylist()
dtmin, dtmax = pa.array([datetime.datetime(1,1,1), datetime.datetime(9999,12,31,23,59,59,999999)], type=pa.timestamp("us")).cast(pa.int64()).to_pylist()

gen_fns = {
    "bigint": ("bigint", pa.int64(), lambda x: min(2**63 * (2 * x - 1), 2**63-1)),
    "int": ("int", pa.uint16(), lambda x: (2**16 - 1) * x),
    "bit": ("bit", pa.bool_(), lambda x: x >= 0.5),
    "decimal": ("decimal(15, 10)", pa.decimal128(15, 10), lambda x: Decimal(((2 * x - 1) * (10**15 - 1)) // 1) / 10**10),
    "float": ("float", pa.float64(), lambda x: (1 if x >= 0.5 else -1) * (x + 1) * 2.0 ** ( 511 * (2 * x - 1))),
    "date": ("date", pa.date32(), lambda x: x * (dmax - dmin) + dmin),
    "time": ("time", pa.time64("us"), lambda x: x * (tmax - tmin) + tmin),
    "datetime2": ("datetime2", pa.timestamp("us"), lambda x: x * (dtmax - dtmin) + dtmin),
    "unicode": ("nvarchar(max)", pa.string(), lambda x: random_utf8(int(x**4 * 4000))),
    "chars_of_length_4000": ("nvarchar(max)", pa.string(), lambda x: random_utf8(4000)),
    "chars_of_length_4001": ("nvarchar(max)", pa.string(), lambda x: random_utf8(4001)),
}

arrays = []
for _, tp, fn in gen_fns.values():
    vals = [
        None if random.random() > 0.75 else
        fn(random.random()) for _ in range(NR_ROWS - 3)
    ]
    vals.insert(random.choice(range(len(vals))), fn(1))
    vals.insert(random.choice(range(len(vals))), fn(0.5))
    vals.insert(random.choice(range(len(vals))), fn(0))
    arrays.append(pa.array(vals, type=tp))

batch_complete = pa.record_batch(arrays, names=list(gen_fns.keys()))

# Set up benchmarking procedure

In [7]:
def setup(cols: list[str]):
    "Reset table in database"
    def f():
        pyodbc_con.execute(f"""
            drop table if exists test
            create table test(
                {", ".join(col + " " + gen_fns[col][0] for col in cols)}
            )
        """)
        pyodbc_con.commit()
    return f

def time_pyodbc(tablock: bool, cols: list[str], n_rows: int):
    batch = batch_complete[:n_rows].select(cols)
    # pyodbc consumes python datatypes, we don't want to time this so we convert ahead of time
    batch_as_tuples = [tuple(row.values()) for row in batch.to_pylist()]

    # pyodbc has the real advantage of being able to cache connections and cursors / statement handles
    # so we don't time the connection setup
    cur = pyodbc_con.cursor()
    cur.fast_executemany = True
    def f():
        cur.executemany(f"""
            insert into test{" with(TABLOCK)" if tablock else ""}
            values ({", ".join("?"*len(cols))})
        """, batch_as_tuples)
        pyodbc_con.commit()
    return f

def time_bcp(tablock: bool, cols: list[str], n_rows: int):
    batch = batch_complete[:n_rows].select(cols)

    def f():
        (bcp_con_tablock if tablock else bcp_con).insert_arrow("test", [batch])
    return f

In [8]:
class Timer:
    "Dummy class to make timeit measure process time and perf counter simultaneously"
    def __init__(self, proc, perf):
        self.proc = proc
        self.perf = perf
    def time():
        return Timer(time.process_time(), time.perf_counter())
    def __sub__(self, other):
        return Timer(self.proc - other.proc, self.perf - other.perf)

In [9]:
def benchmark(columns: list[str], tablock: bool, n_rows: int, all_columns: bool):
    "Perform the benchmark and generate a plotly figure"
    bcp_perf = {}
    bcp_proc = {}
    pyodbc_perf = {}
    pyodbc_proc = {}

    for col in columns:
        t_bcp = timeit.timeit(
            stmt=time_bcp(tablock, [col], n_rows),
            setup=setup([col]),
            timer=Timer.time,
            number=NR_REPS,
        )
        bcp_perf[f"Only {col}"] = t_bcp.perf
        bcp_proc[f"Only {col}"] = t_bcp.proc
        print(f"{col=}: {t_bcp.perf=}, {t_bcp.proc=}")

        t_pyodbc = timeit.timeit(
            stmt=time_pyodbc(tablock, [col], n_rows),
            setup=setup([col]),
            timer=Timer.time,
            number=NR_REPS,
        )

        pyodbc_perf[f"Only {col}"] = t_pyodbc.perf
        pyodbc_proc[f"Only {col}"] = t_pyodbc.proc
        print(f"{col=}: {t_pyodbc.perf=}, {t_pyodbc.proc=}")

    if all_columns:
        t_bcp = timeit.timeit(
            stmt=time_bcp(tablock, columns, n_rows),
            setup=setup(columns),
            timer=Timer.time,
            number=NR_REPS,
        )
        bcp_perf["All columns"] = t_bcp.perf
        bcp_proc["All columns"] = t_bcp.proc
        print(f"all columns: {t_bcp.perf=}, {t_bcp.proc=}")

        t_pyodbc = timeit.timeit(
            stmt=time_pyodbc(tablock, columns, n_rows),
            setup=setup(columns),
            timer=Timer.time,
            number=NR_REPS,
        )

        pyodbc_perf["All columns"] = t_pyodbc.perf
        pyodbc_proc["All columns"] = t_pyodbc.proc
        print(f"all columns: {t_pyodbc.perf=}, {t_pyodbc.proc=}")

    return go.Figure(
        data=[
            go.Bar(name="bcp perf", x=list(bcp_perf.keys()), y=list(bcp_perf.values())),
            go.Bar(name="pyodbc perf", x=list(pyodbc_perf.keys()), y=list(pyodbc_perf.values())),
            go.Bar(name="bcp proc", x=list(bcp_proc.keys()), y=list(bcp_proc.values())),
            go.Bar(name="pyodbc proc", x=list(pyodbc_proc.keys()), y=list(pyodbc_proc.values())),
        ],
        layout=go.Layout(
            height=1000,
            yaxis_title="Time (s)",
        )
    )

# Results

## Normal case

The first benchmark represents a typical workload intended for this module

From the difference between perf/elapsed time and proc/CPU time spent in both bcp and pyodbc benchmarks, we can see that most of the time is not eaten up by the CPU running the notebook. This indicates that the reason pyodbc is slower has nothing to do with the fact that it is using python data types, which are generally slower.

Instead most of the time is spent on the SQL Server side, which, in the case of pyodbc has a very high CPU utilization, as it's trying to figure out how to insert the data and log the inserts.

Having looked at the odbc specification and pyodbc implementation for quite a bit, I think that pyodbc handles inserts about as well as the specification allows.

In [10]:
benchmark(list(gen_fns.keys())[:-2], False, NR_ROWS, True)

col='bigint': t_bcp.perf=3.791813190007815, t_bcp.proc=0.0961361399999987
col='bigint': t_pyodbc.perf=24.702703121991362, t_pyodbc.proc=0.7647799490000011
col='int': t_bcp.perf=3.1573572889901698, t_bcp.proc=0.10439222099999768
col='int': t_pyodbc.perf=24.909668709995458, t_pyodbc.proc=0.801219721999999
col='bit': t_bcp.perf=3.441670886008069, t_bcp.proc=0.1211476989999909
col='bit': t_pyodbc.perf=25.002165749989217, t_pyodbc.proc=0.786839962000002
col='decimal': t_bcp.perf=3.6476841309922747, t_bcp.proc=0.12526347300000396
col='decimal': t_pyodbc.perf=26.832264993019635, t_pyodbc.proc=2.0116444119999954
col='float': t_bcp.perf=3.176042753009824, t_bcp.proc=0.08544665499999837
col='float': t_pyodbc.perf=24.954566051979782, t_pyodbc.proc=0.8603583870000051
col='date': t_bcp.perf=3.4620791990018915, t_bcp.proc=0.08920572499999935
col='date': t_pyodbc.perf=24.832059192005545, t_pyodbc.proc=0.7864785639999923
col='time': t_bcp.perf=3.2379068229929544, t_bcp.proc=0.13379176000000825
col='ti

Now we will do the same benchmark but with a lock on the table.

In [11]:
benchmark(list(gen_fns.keys())[:-2], True, NR_ROWS, True)

col='bigint': t_bcp.perf=1.1818832590070087, t_bcp.proc=0.0768159029999822
col='bigint': t_pyodbc.perf=23.562522699998226, t_pyodbc.proc=0.8834714529999985
col='int': t_bcp.perf=1.124322054994991, t_bcp.proc=0.10225339400000166
col='int': t_pyodbc.perf=22.772727080009645, t_pyodbc.proc=0.9442876520000141
col='bit': t_bcp.perf=1.2112133119953796, t_bcp.proc=0.08725989199999162
col='bit': t_pyodbc.perf=22.80042817501817, t_pyodbc.proc=0.8054483619999928
col='decimal': t_bcp.perf=1.4493887040007394, t_bcp.proc=0.16817898900001182
col='decimal': t_pyodbc.perf=25.375519374996657, t_pyodbc.proc=2.142586217999991
col='float': t_bcp.perf=0.9515895980002824, t_bcp.proc=0.06510844299998553
col='float': t_pyodbc.perf=23.09132196198334, t_pyodbc.proc=0.8196195609999961
col='date': t_bcp.perf=1.1092229799833149, t_bcp.proc=0.09978608000000122
col='date': t_pyodbc.perf=22.930420145014068, t_pyodbc.proc=0.905553824000009
col='time': t_bcp.perf=1.2087761820002925, t_bcp.proc=0.10361756500000752
col='t

As we can see, bulk inserts can use a table lock more effectively than odbc inserts.

## Small Batchsize

The next benchmark demonstrates that when inserting small amounts of rows, the fact that odbc can cache the database connection and bcp needs to reconnect for every bulk insert (as it runs in a subprocess) can be more important than bcp's better performance

In [12]:
benchmark(list(gen_fns.keys())[:-2], False, 1_000, True)

col='bigint': t_bcp.perf=0.34884182500536554, t_bcp.proc=0.008622898999988138
col='bigint': t_pyodbc.perf=0.29327608199673705, t_pyodbc.proc=0.011119887999996081
col='int': t_bcp.perf=0.3800486129766796, t_bcp.proc=0.009693923000014593
col='int': t_pyodbc.perf=0.26780914299888536, t_pyodbc.proc=0.01437000199999261
col='bit': t_bcp.perf=0.35386244900291786, t_bcp.proc=0.011664160999998785
col='bit': t_pyodbc.perf=0.3072272590070497, t_pyodbc.proc=0.006273402000005035
col='decimal': t_bcp.perf=0.3983426060003694, t_bcp.proc=0.013450242999994089
col='decimal': t_pyodbc.perf=0.29824599099811167, t_pyodbc.proc=0.02959976500000039
col='float': t_bcp.perf=0.327064284007065, t_bcp.proc=0.00817542400000093
col='float': t_pyodbc.perf=0.2760500660224352, t_pyodbc.proc=0.007802009999977599
col='date': t_bcp.perf=0.31369118101429194, t_bcp.proc=0.007505693999974028
col='date': t_pyodbc.perf=0.2859841960016638, t_pyodbc.proc=0.01234461300001044
col='time': t_bcp.perf=0.3365580620011315, t_bcp.proc=0

# You win some, you lose some

Sadly bcp did not get blessed with a very good implementation for large data, that is cells with more than 8k bytes. For unicode this translates to 4k characters.

I have tried a number of tweaks to the bcp insert process, including using wide characters instead of utf-8, importing as csv instead of using native datatypes and doing the same using the `BULK IMPORT` statement instead of bcp. However the situation does not improve.

On the plus side, this effect seems to scale linearly with the amount of rows that have cells larger than 8k bytes, so if there are only a few large entries, this will probably not be noticable.

In [13]:
benchmark(list(gen_fns.keys())[-2:], False, 1_000, False)

col='chars_of_length_4000': t_bcp.perf=0.7641439099970739, t_bcp.proc=0.03667466100000638
col='chars_of_length_4000': t_pyodbc.perf=1.4015200669819023, t_pyodbc.proc=0.047185344000013174
col='chars_of_length_4001': t_bcp.perf=9.121761162998155, t_bcp.proc=0.05076896399998532
col='chars_of_length_4001': t_pyodbc.perf=1.1886735489824787, t_pyodbc.proc=0.07327655500000674


# Notes on bandwidth

If network bandwidth is the limiting factor, bcp should also come out ahead by a significant margin. Reasons for that are different data types, for example:

- For both odbc and bcp, each cell gets an indicator variable. With odbc its size is always 8 bytes, with bcp 1 byte is enough, at least for non char/binary types
- Odbc dates send year (2 bytes), month and day (1 byte each) separately, resulting in 4 bytes. bcp uses ordinal dates, so 3 bytes are enough. Same story with datetime
- Odbc often treats character data as utf-16, bcp sends data as utf-8 and lets SQL Server translate. This results in half the bytes in many cases
- Odbc usually sends decimal data as strings because odbc's numeric data type is less reliable. Bcp is sending decimals using integer encoding